In [146]:
from Crypto.Cipher import AES

In [194]:
# AES is 128 bits width which is  128 / 8 = 16  ASCII symbols
BLOCK_SIZE = AES.block_size;
AES_MODE = AES.MODE_ECB;

In [195]:
def xor(block1: list, block2: list) -> list:
    assert(len(block1) == BLOCK_SIZE)
    assert(len(block2) == BLOCK_SIZE)
    return [block1[i] ^ block2[i] for i in range(BLOCK_SIZE)]

### Convert between blocks of uint8 and cypher text hex strings

In [196]:
def hex2block(CTblock: str) -> list:
    assert(len(CTblock) == BLOCK_SIZE * 2)
    block = []
    for i in range(BLOCK_SIZE):
        hx   = CTblock[2*i : 2*i+2]
        block.append(int(hx, 16))
    return block

def block2hex(block: list) -> str:
    assert(len(block) == BLOCK_SIZE)
    CTblock = ''
    for i in range(BLOCK_SIZE):
        CTblock += format(block[i], '02x')
    assert(len(CTblock) == BLOCK_SIZE * 2)
    return CTblock


def CT2blocks(cipher_text: str) -> list:
    assert(len(cipher_text) % (2*BLOCK_SIZE) == 0)
    n_blocks = len(cipher_text) // (2*BLOCK_SIZE)
    blocks = []
    for i in range(n_blocks):
        i1 = 2*BLOCK_SIZE * i
        i2 = 2*BLOCK_SIZE * (i+1)
        CTblock = cipher_text[i1:i2]
        blocks.append(hex2block(CTblock))
    return blocks

def blocks2CT(blocks: list) -> str:
    n_blocks = len(blocks)
    cipher_text = '';
    for i in range(n_blocks):
        cipher_text += block2hex(blocks[i])
    assert(len(cipher_text) % (2*BLOCK_SIZE) == 0)
    return cipher_text

### Convert between blocks of uint8 and plain text strings

In [197]:
def PT2blocks(plain_text: str) -> list:
    # add a padding
    pad_size = BLOCK_SIZE - (len(plain_text) % BLOCK_SIZE)
    padding  = chr(pad_size)
    plain_text += padding * pad_size;
    assert(len(plain_text) % BLOCK_SIZE == 0)
    n_blocks = len(plain_text) // BLOCK_SIZE
    # divide plain text into blocks
    # and encode them to charcodes
    blocks = [];
    for i in range(n_blocks):
        i1 = BLOCK_SIZE * i
        i2 = BLOCK_SIZE * (i+1)
        block = plain_text[i1 : i2];
        codes = [ord(char) for char in block]
        blocks.append(codes)
    return blocks

def blocks2PT(blocks: list) -> str:
    n_blocks = len(blocks)
    assert(n_blocks >= 1)
    plain_text = ''
    for i in range(n_blocks - 1):
        block = blocks[i]
        plain_text += ''.join([chr(code) for code in block])
    padded_block = blocks[-1]
    pad_size = padded_block[-1]
    plain_text += ''.join([chr(code) for code in padded_block[:BLOCK_SIZE - pad_size]])
    return plain_text

In [244]:
class BlockCode:
    def __init__(self, PT_blocks, CT_blocks, algo):
        self.PT_blocks = PT_blocks
        self.CT_blocks = CT_blocks
        assert(algo == 'CBC' or algo == 'CTR')
        self.algo = algo
        
        
    def fromCT(cipher_text, algo):
        CT_blocks = CT2blocks(cipher_text)
        return BlockCode([], CT_blocks, algo)
        
    def fromPT(plain_text, algo):
        PT_blocks = PT2blocks(plain_text)
        return BlockCode(PT_blocks, [], algo)
    
    
    def __encryptCBC(self, iv, key):
        assert(len(self.PT_blocks) != 0)
        assert(isinstance(iv, str) and isinstance(key, str))
        assert(len(key) == BLOCK_SIZE * 2)
        key = bytes(hex2block(key))
        iv  = hex2block(iv)
        M = self.PT_blocks
        n_blocks = len(M)
        C = [iv]
        aes = AES.new(key, AES_MODE)
        for i in range(n_blocks):
            tmp = bytes(xor(C[i], M[i]))
            C.append(list(aes.encrypt(tmp)))
        self.CT_blocks = C
        return blocks2CT(C)
        
    def __decryptCBC(self, key):
        assert(len(self.CT_blocks) != 0)
        assert(isinstance(key, str))
        assert(len(key) == BLOCK_SIZE * 2)
        key = bytes(hex2block(key))
        iv = block2hex(self.CT_blocks[0])
        C  = self.CT_blocks
        n_blocks = len(C) - 1
        M = []
        aes = AES.new(key, AES_MODE)
        for i in range(n_blocks):
            tmp = bytes(C[i+1])
            tmp = list(aes.decrypt(tmp))
            M.append(xor(tmp, C[i]))
        self.PT_blocks = M
        return blocks2PT(M)
    
    
    def __encryptCTR(self, iv, key):
        assert(len(self.PT_blocks) != 0)
        assert(isinstance(iv, str) and isinstance(key, str))
        assert(len(iv) == BLOCK_SIZE * 2 and len(key) == BLOCK_SIZE * 2)
        key = bytes(hex2block(key))
        iv  = int(iv, 16)
        M = self.PT_blocks
        n_blocks = len(M)
        C = [hex2block(format(iv, '032x'))]
        aes = AES.new(key, AES_MODE)
        for i in range(n_blocks):
            tmp = format(iv + i, '032x')
            tmp = bytes(hex2block(tmp))
            tmp = list(aes.encrypt(tmp))
            C.append(xor(M[i], tmp));
        self.CT_blocks = C
        return blocks2CT(C)
        
    def __decryptCTR(self, key):
        assert(len(self.CT_blocks) != 0)
        assert(isinstance(key, str))
        assert(len(key) == BLOCK_SIZE * 2)
        key = bytes(hex2block(key))
        iv = block2hex(self.CT_blocks[0])
        iv = int(iv, 16)
        C  = self.CT_blocks[1:]
        n_blocks = len(C)
        M = []
        aes = AES.new(key, AES_MODE)
        for i in range(n_blocks):
            tmp = format(iv + i, '032x')
            tmp = bytes(hex2block(tmp))
            tmp = list(aes.encrypt(tmp))
            M.append(xor(C[i], tmp));
        padded_block = C[-1]
        pad_size = -1
        for i in range(BLOCK_SIZE):
            if pad_size == -1 and padded_block[i] == 0:
                pad_size = BLOCK_SIZE - i
            if pad_size > 0:
                M[-1][i] = pad_size
        self.PT_blocks = M
        return blocks2PT(M)
        
        
    def encrypt(self, iv, key):
        if self.algo == 'CBC':
            return self.__encryptCBC(iv, key)
        else:
            return self.__encryptCTR(iv, key)
        
    def decrypt(self, key):
        if self.algo == 'CBC':
            return self.__decryptCBC(key)
        else:
            return self.__decryptCTR(key)

In [245]:
obj1 = BlockCode.fromCT(
    '4ca00ff4c898d61e1edbf1800618fb2828a226d160dad07883d04e008a7897ee2e4b7465d5290d0c0e6c6822236e1daafb94ffe0c5da05d9476be028ad7c1d81', 
    'CBC'
)
m = obj1.decrypt('140b41b22a29beb4061bda66b6747e14')
print(m)

Basic CBC mode encryption needs padding.


In [246]:
obj1 = BlockCode.fromCT(
    '5b68629feb8606f9a6667670b75b38a5b4832d0f26e1ab7da33249de7d4afc48e713ac646ace36e872ad5fb8a512428a6e21364b0c374df45503473c5242a253', 
    'CBC'
)
m = obj1.decrypt('140b41b22a29beb4061bda66b6747e14')
print(m)

Our implementation uses rand. IV


In [247]:
msg = '69dda8455c7dd4254bf353b773304eec0ec7702330098ce7f7520d1cbbb20fc388d1b0adb5054dbd7370849dbf0b88d393f252e764f1f5f7ad97ef79d59ce29f5f51eeca32eabedd9afa9329'
msg += '00'*4
obj1 = BlockCode.fromCT(
    msg, 
    'CTR'
)
m = obj1.decrypt('36f18357be4dbd77f050515c73fcf9f2')
print(m)

CTR mode lets you build a stream cipher from a block cipher.


In [248]:
msg = '770b80259ec33beb2561358a9f2dc617e46218c0a53cbeca695ae45faa8952aa0e311bde9d4e01726d3184c34451'
msg += '00'*2
obj1 = BlockCode.fromCT(
    msg, 
    'CTR'
)
m = obj1.decrypt('36f18357be4dbd77f050515c73fcf9f2')
print(m)

Always avoid the two time pad!


In [249]:
msg = 'Hello, World! a98 *&230(*^E)...aofsdhiosoadiuoi asodghi df.  igdufgh eoru soijd oaiwy9498ty odfgo[asjdpoj ]'
key = '36f18357be4dbd77f050515c73fcf9f2'
iv  = '140b41b22a29beb4061bda66b6747e14'

cod = BlockCode.fromPT(msg, 'CBC')
C = cod.encrypt(iv, key)
dec = BlockCode.fromCT(C, 'CBC')
print(dec.decrypt(key))

Hello, World! a98 *&230(*^E)...aofsdhiosoadiuoi asodghi df.  igdufgh eoru soijd oaiwy9498ty odfgo[asjdpoj ]


In [250]:
msg = 'Hello, World! a98 *&230(*^E)...aofsdhiosoadiuoi asodghi df.  igdufgh eoru soijd oaiwy9498ty odfgo[asjdpoj ]'
key = '36f18357be4dbd77f050515c73fcf9f2'
iv  = '140b41b22a29beb4061bda66b6747e14'

cod = BlockCode.fromPT(msg, 'CTR')
C = cod.encrypt(iv, key)
dec = BlockCode.fromCT(C, 'CTR')
print(dec.decrypt(key))

Hello, World! a98 *&230(*^E)...aofsdhiosoadiuoi asodghi df.  igdufgh eoru soijd oaiwy9498ty odfgo[asjdpoj ]
